Realizando importações necessárias para a implementação do Spark + Delta Lake

In [9]:
import pyspark
import os
from delta import *
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import *

Criando uma sessão do Spark com Delta

In [10]:
builder = pyspark.sql.SparkSession.builder.appName("DeltaLake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

Criando um contexto no Spark

In [11]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Criando a tabela "climate-change" no formato delta

In [16]:
print("Iniciando criação da tabela...")

root = os.path.dirname(os.getcwd())

climate_change_df = spark.read.parquet("{0}/datasource/climate_change.parquet".format(root))
climate_change_df.write.mode(saveMode="overwrite").save(path="data/climate_change", format="delta")

Iniciando criação da tabela...


Lendo informações da tabela criada

In [ ]:
cc_df = spark.read.load("data/climate_change", format="delta")


cc_df.filter((cc_df["City"] == "Porto Alegre") & (cc_df["Country"] == "Brazil")).show(5)

Update na tabela

In [ ]:
cc_table = DeltaTable.forPath(spark, "data/climate_change")

cc_table.update(
    condition=expr("City == 'Porto Alegre'"),
    set={"Country": lit("Exemplo Update")})

cc_df = cc_table.toDF()
cc_df.filter(cc_df["City"] == "Porto Alegre").show(5)

Delete na tabela

In [ ]:
cc_table = DeltaTable.forPath(spark, "data/climate_change")

cc_table.delete(expr("City == 'Porto Alegre'"))

cc_df = cc_table.toDF()
cc_df.filter(cc_df["City"] == "Porto Alegre").show(5)

Lendo informações antigas da tabela

In [30]:
print("Tabela anterior a atualização")
df_zero = spark.read.format("delta").option("versionAsOf", 0).load("data/climate_change")
df_zero.filter(df_zero["City"] == "Porto Alegre").show(5)

print('\n')
print('\n')

print("Tabela após a atualização e anterior ao delete")
df_um = spark.read.format("delta").option("versionAsOf", 1).load("data/climate_change")
df_um.filter(df_um["City"] == "Porto Alegre").show(5)

print('\n')
print('\n')

print("Tabela após o delete")
df_dois = spark.read.format("delta").option("versionAsOf", 2).load("data/climate_change")
df_dois.filter(df_dois["City"] == "Porto Alegre").show(5)

Tabela anterior a atualização
+----------+------------------+-----------------------------+------------+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|        City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+------------+-------+--------+---------+
|1832-01-01|            23.173|                        1.692|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-02-01|             22.93|                        2.661|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-03-01|            21.281|                        1.826|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-04-01|            17.697|                        2.482|Porto Alegre| Brazil|  29.74S|   51.69W|
|1832-05-01|            14.214|                        2.263|Porto Alegre| Brazil|  29.74S|   51.69W|
+----------+------------------+-----------------------------+------------+-------+--------+---------+
only showing top 5 rows





Tabela após a atualizaç

+----------+------------------+-----------------------------+------------+--------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|        City|       Country|Latitude|Longitude|
+----------+------------------+-----------------------------+------------+--------------+--------+---------+
|1832-01-01|            23.173|                        1.692|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-02-01|             22.93|                        2.661|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-03-01|            21.281|                        1.826|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-04-01|            17.697|                        2.482|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
|1832-05-01|            14.214|                        2.263|Porto Alegre|Exemplo Update|  29.74S|   51.69W|
+----------+------------------+-----------------------------+------------+--------------+--------+---------+
only showing top 5 